In [7]:
import pandas as pd
import numpy as np

data = pd.read_csv('sample_data.csv')
label = pd.read_csv('sample_label.csv')
print(data.shape)
print(label['category'][1:].shape)

(499, 4096)
(499,)


In [8]:
from keras.layers import Embedding, ELU, Dropout, LSTM, Flatten, Input, Dense, BatchNormalization, SpatialDropout1D
from keras.regularizers import l2, l1_l2
from keras.optimizers import Adam
from keras import Model
import matplotlib.pyplot as plt
import pandas as pd

maxlen=4096
embed_size=16

main_input = Input(shape=(maxlen,), name='main_input')
emb = Embedding(256, embed_size, input_length=maxlen, embeddings_regularizer=l2(1e-4))(main_input)
drop = SpatialDropout1D(0.2)(emb)
hs = [] #hidden states from each LSTM layer stored here 
hs.append(LSTM(S, dropout_W=0.5, dropout_U=0.5, W_regularizer=l2(1e-5), U_regularizer=l2(1e-5), return_sequences=True)(drop))
for l in range(1, num_layers):
    hs.append(LSTM(S, dropout_W=0.5, dropout_U=0.5, W_regularizer=l2(1e-5), U_regularizer=l2(1e-5), return_sequences=True)(hs[-1]))
local_states = merge(hs, mode='concat')
average_active = AverageAcrossTime()(local_states) # this produces h
state_size = lstm_layer_size*num_layers 

# Attention mechanism starts here 
attn_cntx = merge([local_states, RepeatVector(maxlen)(average_active)], mode='concat')
attn_cntx = TimeDistributed(Dense(lstm_layer_size, activation='linear', W_regularizer=l2(1e-4)))(attn_cntx) 
attn_cntx = TimeDistributed(BatchNormalization())(attn_cntx) 
attn_cntx = TimeDistributed(Activation('tanh'))(attn_cntx) 
attn_cntx = TimeDistributed(Dropout(0.5))(attn_cntx) 

attn = TimeDistributed(Dense(1, activation='linear', W_regularizer=l2(1e-4)))(attn_cntx)
attn = Flatten()(attn)
attn = Activation('softmax')(attn)
attn = Reshape((maxlen, 1))(attn)
attn = TileOut(state_size)(attn) #repeats value to make a specific shape

final_context = merge([attn, local_states], mode='mul') 
final_context = SumAcrossTime()(final_context) # eq(2), Ti=1 αihi
final_context = Dense(state_size, activation='linear', W_regularizer=l2(1e-4))(final_context) 
final_context = BatchNormalization()(final_context) 
final_context = Activation('tanh')(final_context) 
final_context = Dropout(0.5)(final_context) 

loss_out = Dense(1, activation='sigmoid', name='loss_out')(final_context)
model = Model(inputs=[main_input], outputs=[loss_out]) 
optimizer = Adam(lr=0.001, clipnorm=1.0) 
model.compile(optimizer, loss='binary_crossentropy')

model.fit(data, label['category'][1:], epochs=1, verbose=1)

NameError: name 'S' is not defined